In [1]:
import psycopg2
import pandas as pd
import uuid

In [2]:
dbhost = "localhost"
dbname = "db_02"
dbuser = "postgres"
dbpassword = "ABcd@12#$"

# 1/4 train, dev and test difference
# source = 'creditcard_train'
# source = 'creditcard_dev'
source = 'creditcard_test'

In [3]:
file_data = pd.read_csv(f"C:/Users/979ar/OneDrive/Desktop/Minor Project/{source}.csv")
# print(len(file_data))

try:
    conn = psycopg2.connect(dbname=dbname, user=dbuser, password=dbpassword, host=dbhost)
except psycopg2.Error as e:
    print("Error connecting to database:", e)
    exit()

if(conn):
    print('connected')

cur = conn.cursor()

connected


In [4]:
Arr=[]
for i in range (1, 29):
    Arr.append(f"V{i}")

columns = ", ".join(Arr)
creation_columns=" DECIMAL, ".join(Arr)

In [5]:
# cur.execute(f"DROP TABLE {source};")

# cur.execute(f"SELECT * FROM {source};")
# for row in cur.fetchall():
#     print(row, end='\n')

# 2/4 train, dev and test difference
# cur.execute(f"CREATE TABLE {source} (Time INTEGER, {creation_columns} DECIMAL, Amount INTEGER, Class INTEGER);") #train and dev
cur.execute(f"CREATE TABLE {source} (ID CHAR(36), Time INTEGER, {creation_columns} DECIMAL, Amount INTEGER);") #test

# 3/4 train, dev and test difference
# insert_query = f"INSERT INTO {source} (Time, {columns}, Amount, Class) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);" #train and dev
insert_query = f"INSERT INTO {source} (ID, Time, {columns}, Amount) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);" #test

In [6]:
def toSql(dataToSend):
    try:
        cur.execute(insert_query, dataToSend)
        conn.commit()
    except psycopg2.Error as e:
        print("Error inserting data:", e)

In [7]:
data=[]

for i in range(0, len(file_data)):
    temp=file_data.iloc[i]
    # print(temp)
    t=[]
    # 4/4 train, dev and test difference
    # for j in range (0, 31): #train and dev
    t.append(str(uuid.uuid4()))
    for j in range (0, 30): #test
        t.append(temp[j])
    
    toSql(t)
print('done')
#creditcard_train 6m 11.5s :)
#creditcard_dev m 55.6s ...
#creditcard_test 55.9s

C:\Users\979ar\AppData\Local\Temp\ipykernel_21720\351362125.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  t.append(temp[j])


done


In [8]:
# cur.execute(f"SELECT ID FROM {source};")
# for row in cur.fetchall():
#     print(row, end='\n')

In [9]:
conn.commit()
cur.close()
conn.close()